# Import Libraries

In [134]:
import pandas as pd
import re
import mysql.connector
from sqlalchemy import create_engine
import json

# Preparing Data

Thanks to the previous EDA performed on the datasets, we have a clearer idea of what is needed to make a more effective merge.

In [135]:
df_data = pd.read_csv('../data/clean_data.csv')
df_api = pd.read_csv('../data/dataApi.csv')

### Preparing API Data

In [136]:
columns_to_drop = ['photos', 'primaryHost', 'additionalHosts', 'isHostedBySuperhost', 'calendar', 'url', 'id', 'location', 'reviews','isAvailable']
df_api = df_api.drop(columns=columns_to_drop)
df_data = df_data.drop(columns=['month.1'])

In [137]:
def extract_amount(text):
    match = re.search(r"'amount'\s*:\s*(\d+)", text)
    return int(match.group(1)) if match else None
    
df_api['amount'] = df_api['pricing'].astype(str).apply(extract_amount)
df_api = df_api.drop(columns=['pricing'])

In [138]:
df_api.rename(columns={
    'numberOfGuests': 'guests',
    'amount': 'nightly_rate',
    'address': 'city',
    'occupancyPercentage': 'openness'
}, inplace=True)
df_api

,name,stars,guests,city,roomType,openness,nightly_rate
0,Dreamy Joshua Tree Hideaway,4.94,2,"Yucca Valley, California, United States",Entire home,52.63,143
1,"""Ironwood Cabin"" A cozy escape to Lake Arrowhead!",4.95,6,"Lake Arrowhead, California, United States",Entire cabin,31.58,146
2,heavenly valley lodge,4.92,8,"Big Bear Lake, California, United States",Entire cabin,36.95,1141
3,heavenly valley lodge,4.92,8,"Big Bear Lake, California, United States",Entire cabin,0.00,1141
4,Las Palmas: Desert Oasis w/ Hot Tub & Greenhouse,5.00,5,"Yucca Valley, California, United States",Entire home,84.21,183
...,...,...,...,...,...,...,...
3877,"Close to the slopes! Kings,bunks,gameroom,& vi...",4.88,14,"Big Bear Lake, California, United States",Entire home,0.00,458
3878,RockHill Ranch-Joshua Tree Retreat,5.00,4,"Joshua Tree, California, United States",Entire home,63.16,328
3879,Carrera Cozy Cabin: FencedBackyard+FirePit+BBQ,5.00,8,"Big Bear, California, United States",Entire cabin,5.26,149
3880,Carrera Cozy Cabin: FencedBackyard+FirePit+BBQ,5.00,8,"Big Bear, California, United States",Entire cabin,0.59,149


In [139]:
df_api['city'] = df_api['city'].apply(lambda x: x.split(',')[0])
df_api['guests'] = df_api['guests'].astype(float)
df_api['nightly_rate'] = df_api['nightly_rate'].astype(float)
df_api['nightly_rate'] = df_api['nightly_rate'].round(2)
df_api['openness'] = df_api['openness'].round(2)
df_data['nightly_rate'] = df_data['nightly_rate'].round(2)
df_data['openness'] = df_data['openness'].round(2)


In [140]:
merged_df = pd.merge(df_api, df_data, on=['city', 'guests', 'nightly_rate', 'openness'], how='outer')
merged_df.head()

,name,stars,guests,city,roomType,openness,nightly_rate,month,hot_tub,pool,host_type,bedrooms,bathrooms,revenue,occupancy,lead_time,length_stay
0,Brand New Custom Built Modern Lake/Ski House,4.98,10.0,Big Bear Lake,Entire home,0.0,571.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brand New Custom Built Modern Lake/Ski House,4.98,10.0,Big Bear Lake,Entire home,4.4,571.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cheerful 2 bdrm home w/fireplace in the mounta...,4.96,4.0,Angelus Oaks,Entire cabin,0.0,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cheerful 2 bdrm home w/fireplace in the mounta...,4.96,4.0,Angelus Oaks,Entire cabin,0.0,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Romantic & Cozy A-Frame Getaway! 7 min to Ski,4.93,4.0,Arrowbear,Entire place,39.0,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
null_percentages = merged_df.isnull().mean() * 100
null_percentages

name            88.518190
stars           88.663117
guests           0.000000
city             0.000000
roomType        88.518190
openness         0.000000
nightly_rate     0.000000
month           11.481810
hot_tub         11.481810
pool            11.481810
host_type       11.481810
bedrooms        11.481810
bathrooms       11.481810
revenue         11.481810
occupancy       11.481810
lead_time       11.481810
length_stay     11.481810
dtype: float64

## Null Data Handling & Imputation

In [142]:
for column in ['month', 'host_type']:
    mode_value = merged_df[column].mode()[0]
    merged_df[column] = merged_df[column].fillna(mode_value)

for column in ['hot_tub', 'pool', 'bedrooms', 'bathrooms', 'revenue', 'occupancy', 'lead_time', 'length_stay']:
    median_value = merged_df[column].median()
    merged_df[column] = merged_df[column].fillna(median_value)


In [143]:
merged_df['name'] = merged_df['name'].fillna('Unknown')
merged_df['roomType'] = merged_df['roomType'].fillna('Unknown')

merged_df['stars'] = merged_df['stars'].fillna(-999)

### Connection and Load to DB

In [145]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import json

with open('../config/credentials.json', 'r') as json_file:
    credentials = json.load(json_file)

connection = mysql.connector.connect(
    host=credentials['host'],
    user=credentials['user'],
    password=credentials['password'],
    database=credentials['database']  
)

cursor = connection.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS etl")
cursor.execute("USE etl")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS merged_data (
        name VARCHAR(255),
        stars FLOAT,
        guests FLOAT,
        city VARCHAR(100),
        roomType VARCHAR(100),
        openness FLOAT,
        nightly_rate FLOAT,
        month VARCHAR(50),
        hot_tub FLOAT,
        pool FLOAT,
        host_type VARCHAR(100),
        bedrooms FLOAT,
        bathrooms FLOAT,
        revenue FLOAT,
        occupancy FLOAT,
        lead_time FLOAT,
        length_stay FLOAT
    )
""")

cursor.close()
connection.close()

engine = create_engine(f"mysql+mysqlconnector://{credentials['user']}:{credentials['password']}@{credentials['host']}/etl")
merged_df.to_sql(name='merged_data', con=engine, if_exists='replace', index=False)


33810